In [16]:
if False:
    %pip install langchain_community
    %pip install langchain_core
    %pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 2.5 MB/s eta 0:00:002.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.8 MB/s eta 0:00:00m eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.


## this just tests whether our connection works

In [7]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Local Llama3 
llm = ChatOllama(
    model="llama3",
    keep_alive=-1, # keep the model loaded indefinitely
    temperature=0,
    max_new_tokens=512)

prompt = ChatPromptTemplate.from_template("Write me a 500 word article on {topic} from the perspective of a {profession}. ")

# using LangChain Expressive Language chain syntax
chain = prompt | llm | StrOutputParser()

print(chain.invoke({"topic": "LLMs", "profession": "shipping magnate"}))



**"LLMs: The Game-Changers for Shipping Magnates Like Me"**

As a shipping magnate, I've spent my fair share of time navigating the complexities of global trade and commerce. From managing fleets of vessels to negotiating with ports and customs officials, there's no shortage of challenges in this industry. But one development that has caught my attention - and potentially changed the game for me and my competitors alike - is the rise of Large Language Models (LLMs).

At first glance, LLMs might seem like a distant cousin to the world of shipping. After all, they're AI-powered language processing systems designed to analyze and generate human-like text. But trust me, their impact on our industry has been nothing short of profound.

Let's start with the obvious: data analysis. As a shipping magnate, I need to stay on top of market trends, demand patterns, and supply chain disruptions. LLMs have revolutionized my ability to do just that. By processing vast amounts of text-based data - fro

### And this shows the same as above - but using streaming instead of waiting for completed inference.

In [8]:
for chunk in chain.stream({"topic": "pizza", "profession": "mushroom"}):
    print(chunk, end="", flush=True)

The life of a mushroom on a pizza is a good one. I've had my fair share of adventures, and let me tell you, it's been a wild ride. From the damp earth to the scorching hot oven, I've seen it all.

It starts with the humble beginnings, growing in the dark, damp soil. It's a slow process, but eventually, I emerge as a tiny little thing, ready to take on the world. Or at least, that's what I thought. Little did I know, my fate was sealed the moment I was plucked from the earth and placed onto a pizza.

The first thing I notice is the aroma. It's intoxicating, a mix of melted cheese, savory sauce, and spices that make my little mushroom head spin. I'm not sure what it is about the smell of pizza, but it's like nothing else in the world. And then there are the toppings - the gooey mozzarella, the crispy pepperoni, the tangy olives... it's a sensory overload.

But as much as I love the aroma and the toppings, the real excitement comes when the pizza is placed into the oven. That's when thing

### Now let's get some JSON back

In [14]:
import json
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

json_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {
            "title": "Name",
            "description": "The person's name",
            "type": "string"
        },
        "age": {
            "title": "Age",
            "description": "The person's age",
            "type": "integer",
            "minimum": 0
        },
        "favorite_food": {
            "title": "Favorite Food",
            "description": "The person's favorite food",
            "type": "string"
        }
    },
    "required": ["name", "age", "favorite_food"]
}

llm = ChatOllama(
    model="llama3",
    format="json",
    keep_alive=-1, # keep the model loaded indefinitely
    temperature=0.1,
    max_new_tokens=512
    )

messages = [
    HumanMessage(
        content="Please tell me about a person using the following JSON schema:"
    ),
    HumanMessage(content="{schema}"),
    HumanMessage(
        content="Now, considering the schema, tell me about a person named John who is 35 years old and loves pizza."
    ),
]

prompt = ChatPromptTemplate.from_messages(messages)

#converting the json schema to a string
dumps = json.dumps(json_schema, indent=2)


chain = prompt | llm | JsonOutputParser()



response = chain.invoke({"schema": dumps})
print(json.dumps(response, indent=4))
print(type(response))

"{\"name\": \"John\", \"age\": 35, \"hobbies\": [\"pizza\"]}\n\n  \n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  "
<class 'str'>


Note that if you don't use JsonOutputParser we get a string instead of a dict


In [15]:
chain = prompt | llm | StrOutputParser()
print(json.dumps(response, indent=4))
print(type(response))

"{\"name\": \"John\", \"age\": 35, \"hobbies\": [\"pizza\"]}\n\n  \n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n  "
<class 'str'>


let's look at using the actul function calling / structured responses support in ollama 



In [38]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions

# Pydantic Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")

context = """Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde."""

# Prompt template llama3
prompt = PromptTemplate.from_template(
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a smart assistant take the following context and question below and return your answer in JSON.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
QUESTION: {question} \n
CONTEXT: {context} \n
JSON:
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
 """
)

# Chain
llm = OllamaFunctions(model="llama3", 
                      format="json", 
                      temperature=0)

structured_llm = llm.with_structured_output(Person)
chain = prompt | structured_llm

response = chain.invoke({
    "question": "Who is taller?",
    "context": context
    })

print(response)

name='Claudia' height=6.0 hair_color='brunette'


now it's the same thing but phi3 - only real diff is the prompt template

however - it may output a pydantic error by not stickign to the tempalte - if that happens we need to try again with a variation 


In [40]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from pydantic import ValidationError

# Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")

context = """Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde."""


# Prompt template phi 3
prompt = PromptTemplate.from_template(
    """<|user|>{context}

QUESTION: {question}<|end|>
<|assistant|>AI: """
)

# Function to adjust temperature and retry
def invoke_with_adjusted_temperature(context, question, initial_temperature, max_attempts=10):
    temperature = initial_temperature
    attempts = 0

    while attempts < max_attempts:
        try:
            print (f"attempt: {attempts}")
            llm = OllamaFunctions(model="phi3", format="json", temperature=temperature)
            structured_llm = llm.with_structured_output(Person)
            chain = prompt | structured_llm
            
            response = chain.invoke({
                "question": question,
                "context": context
            })
            print (f"this took {attempts} attempts")
            return response  # Successful parsing
        except Exception as e:
            print(f"Attempt {attempts + 1}: Failed to parse with temperature {temperature}. Trying again...")
            attempts += 1
            temperature += 0.1  # Increment temperature
            
            if attempts == max_attempts:
                print("Maximum attempts reached. Raising the last error encountered.")
                raise e  # Raise the last error after max attempts

# Call the function
try:
    response = invoke_with_adjusted_temperature(context, "Who is taller?", initial_temperature=0)
    print(response)
except Exception as e:
    print("Final error:", e)


attempt: 0
Attempt 1: Failed to parse with temperature 0. Trying again...
attempt: 1
Attempt 2: Failed to parse with temperature 0.1. Trying again...
attempt: 2
Attempt 3: Failed to parse with temperature 0.2. Trying again...
attempt: 3
Attempt 4: Failed to parse with temperature 0.30000000000000004. Trying again...
attempt: 4
Attempt 5: Failed to parse with temperature 0.4. Trying again...
attempt: 5
Attempt 6: Failed to parse with temperature 0.5. Trying again...
attempt: 6
this took 6 attempts
name='Claudia' height=6.0 hair_color='brunette'


a random guy said the prompt format was wrong 

In [41]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from pydantic import ValidationError

# Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")

context = """Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde."""


# Prompt template phi 3 - from some guy
prompt = PromptTemplate.from_template(
    """{context}
    Human: {question}
    AI:"""
)

# Function to adjust temperature and retry
def invoke_with_adjusted_temperature(context, question, initial_temperature, max_attempts=10):
    temperature = initial_temperature
    attempts = 0

    while attempts < max_attempts:
        try:
            print (f"attempt: {attempts}")
            llm = OllamaFunctions(model="phi3", format="json", temperature=temperature)
            structured_llm = llm.with_structured_output(Person)
            chain = prompt | structured_llm
            
            response = chain.invoke({
                "question": question,
                "context": context
            })
            print (f"this took {attempts} attempts")
            return response  # Successful parsing
        except Exception as e:
            print(f"Attempt {attempts + 1}: Failed to parse with temperature {temperature}. Trying again...")
            attempts += 1
            temperature += 0.1  # Increment temperature
            
            if attempts == max_attempts:
                print("Maximum attempts reached. Raising the last error encountered.")
                raise e  # Raise the last error after max attempts

# Call the function
try:
    response = invoke_with_adjusted_temperature(context, "Who is taller?", initial_temperature=0)
    print(response)
except Exception as e:
    print("Final error:", e)


attempt: 0
this took 0 attempts
name='Claudia' height=6.0 hair_color='brunette'


let's do function calling

In [30]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage

model = OllamaFunctions(
    model="llama3", 
    format="json"
    )

model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

response = model.invoke("what is the weather in Singapore?")

print(response)

content='' additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Singapore", "unit": "celsius"}'}} id='run-71389e7d-2d14-4107-9aa0-ed84d064a7c2-0'


and same in phi

In [31]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage

model = OllamaFunctions(
    model="phi3", 
    keep_alive=-1,
    format="json"
    )

model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

response = model.invoke("what is the weather in Singapore?")

print(response)

content='' additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Singapore"}'}} id='run-ad611cdb-de60-43e2-a00e-b28b6fc4dea0-0'
